# Ensemble Model Training - Simplified Single-Branch Models

This notebook trains separate simplified CNN models for each feature type and then creates an ensemble.

In [1]:
from datetime import datetime
import os
import json
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.utils import to_categorical, Sequence
from keras.models import Model
from keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
)
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from modules.PostgresDBHandler import PostgresDBHandler
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam

2025-07-07 18:04:21.014326: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-07 18:04:21.174242: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-07 18:04:21.174318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-07 18:04:21.198741: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-07 18:04:21.250971: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Configuration
dbParams = {
    "dbname": "mydatabase",
    "user": "myuser",
    "password": "mypassword",
    "host": "postgres_server",
    "port": "5432",
}

EPOCHS = 200
BATCH_SIZE = 32
KFOLD_SPLITS = 5
FIXED_LENGTH = 128

# Feature types to train models for
FEATURE_TYPES = [
    'mel_spectrogram', 'mfcc', 'chromagram', 'spectral_contrast',
    'tonnetz', 'constant_q', 'cqt', 'stft', 'harmonic_percussive', 'onset_strength'
]

# GPU configuration
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Number of available GPUs: {len(gpus)}")
    except RuntimeError as e:
        print(e)

2025-07-07 18:04:22.658424: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-07 18:04:22.682989: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# Initialize database connection
db = PostgresDBHandler(**dbParams)
db.connect()

# Get instrument mappings
instruments_mappings = db.get_mappings_instruments()
num_classes = len(instruments_mappings)
print(f"Number of instrument classes: {num_classes}")
print("Instruments:", instruments_mappings['name'].tolist())

db.close()

Number of instrument classes: 9
Instruments: ['clarinet', 'piccolo', 'cello', 'oboe', 'violin', 'flute', 'bass', 'trumpet', 'sax']


In [4]:
dbConnect = PostgresDBHandler(**dbParams)
dbConnect.connect()
audioIDs = dbConnect.get_all_unique_audio_ids_in_processed()
processed_data = dbConnect.get_processed_fit_data(audioIDs)

all_processed_data = []
for audio_id in audioIDs:
    features = dbConnect.get_all_feature_types_for_audio(audio_id)
    feature_dict = {f['featureTypeName']: f['featurePath'] for f in features}
    instrumentID = dbConnect.get_audio_file(audio_id)['instrumentID']
    feature_dict['instrumentID'] = instrumentID
    all_processed_data.append(feature_dict)

dbConnect.close()

In [5]:
processed_df = pd.DataFrame(all_processed_data)
processed_df

,mel_spectrogram,mfcc,chromagram,spectral_contrast,tonnetz,constant_q,cqt,stft,harmonic_percussive,onset_strength,instrumentID
0,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/9dfbbb55-f9...,ensemble_intermediate_results/chromagram/e7df2...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/f3d964d2...,ensemble_intermediate_results/constant_q/41991...,ensemble_intermediate_results/cqt/30d6acbd-9f7...,ensemble_intermediate_results/stft/816fb586-fb...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/1...,5
1,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/bcf73ea3-4a...,ensemble_intermediate_results/chromagram/7b11a...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/834affe9...,ensemble_intermediate_results/constant_q/cbdfe...,ensemble_intermediate_results/cqt/3f142ac1-ea2...,ensemble_intermediate_results/stft/033f004c-7e...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/3...,8
2,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/457fe528-fd...,ensemble_intermediate_results/chromagram/63dd9...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/304decf7...,ensemble_intermediate_results/constant_q/56f27...,ensemble_intermediate_results/cqt/7002caaa-566...,ensemble_intermediate_results/stft/99975081-8d...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/1...,2
3,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/a88d64a3-5f...,ensemble_intermediate_results/chromagram/7a5a7...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/c96baa79...,ensemble_intermediate_results/constant_q/c0b17...,ensemble_intermediate_results/cqt/b64fa305-b26...,ensemble_intermediate_results/stft/9d8360f8-f9...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/6...,4
4,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/bd978511-62...,ensemble_intermediate_results/chromagram/1690d...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/220438d2...,ensemble_intermediate_results/constant_q/83d09...,ensemble_intermediate_results/cqt/5f929abc-8f0...,ensemble_intermediate_results/stft/edd8790a-14...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/b...,2
...,...,...,...,...,...,...,...,...,...,...,...
895,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/5ecd7480-ba...,ensemble_intermediate_results/chromagram/c137c...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/3ab4fd1c...,ensemble_intermediate_results/constant_q/0d54d...,ensemble_intermediate_results/cqt/dc36f0a8-229...,ensemble_intermediate_results/stft/d5dd41f4-15...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/6...,6
896,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/abc22b4f-98...,ensemble_intermediate_results/chromagram/78f2e...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/cdc7b184...,ensemble_intermediate_results/constant_q/5eb48...,ensemble_intermediate_results/cqt/19bd3542-407...,ensemble_intermediate_results/stft/ac8137f2-8d...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/3...,6
897,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/adb16d86-26...,ensemble_intermediate_results/chromagram/ee1bf...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/0e83ec33...,ensemble_intermediate_results/constant_

In [ ]:
def get_input_shape(feature_type, df):
    feature_path_col = feature_type 
    for path in df[feature_path_col]:
        if isinstance(path, str) and os.path.exists(path):
            arr = np.load(path)
            return arr.shape
    raise ValueError(f"No valid file found for {feature_type}")

In [7]:
class SingleFeatureDataGenerator(Sequence):
    def __init__(self, df, feature_col, batch_size=32, shuffle=True, num_classes=None):
        self.df = df.reset_index(drop=True)
        self.feature_col = feature_col
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_classes = num_classes
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def on_epoch_end(self):
        self.indices = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indices]

        X = []
        y = []

        for _, row in batch_df.iterrows():
            try:
                arr = np.load(row[self.feature_col])
            except Exception as e:
                print(f"Error loading {row[self.feature_col]}: {e}")
                continue
        
            if np.isnan(arr).any() or np.isinf(arr).any():
                raise ValueError(f"Feature file {row[self.feature_col]} contains NaNs or Infs.")
        
            arr = (arr - np.mean(arr)) / (np.std(arr) + 1e-8)
            if arr.ndim == 2:
                arr = np.expand_dims(arr, -1)  # shape: (H, W, 1)
        
            X.append(arr)
            y.append(row['instrumentID'])  # already label-encoded
        
        X = np.array(X)
        y = to_categorical(np.array(y), num_classes=self.num_classes)
        
        return X, y

In [12]:
def create_simple_model(input_shape, num_classes, model_name="simple_cnn"):
    input_layer = Input(shape=(*input_shape, 1), name=f"{model_name}_input")

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)

    x = Dense(16, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    output = Dense(num_classes, activation='softmax', name=f"{model_name}_output")(x)

    model = Model(inputs=input_layer, outputs=output, name=model_name)
    return model


In [13]:
results = {}

for feature_type in tqdm(FEATURE_TYPES, desc="Training features"):
    print(f"\n{'='*40}\nTraining model for {feature_type}\n{'='*40}")

    feature_col = feature_type
    feature_df = processed_df.dropna(subset=[feature_col])
    
    # Global label encoder
    label_encoder = LabelEncoder()
    label_encoder.fit(feature_df['instrumentID'])

    feature_df = feature_df.copy()
    feature_df['instrumentID'] = label_encoder.transform(feature_df['instrumentID'])
    num_classes = len(label_encoder.classes_)
    input_shape = get_input_shape(feature_type)

    kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42)
    accuracy_list, loss_list, history_list = [], [], []
    classification_reports, confusion_matrices = [], []

    for fold, (train_idx, test_idx) in enumerate(kf.split(feature_df)):
        print(f"\n--- Fold {fold+1}/{KFOLD_SPLITS} ---")
        train_df = feature_df.iloc[train_idx].reset_index(drop=True)
        test_df = feature_df.iloc[test_idx].reset_index(drop=True)

        train_df, val_df = train_test_split(
            train_df, test_size=0.2, random_state=42, stratify=train_df['instrumentID'])

        # Generators (labels are already encoded)
        train_gen = SingleFeatureDataGenerator(train_df, feature_col, BATCH_SIZE, shuffle=True, num_classes=num_classes)
        val_gen   = SingleFeatureDataGenerator(val_df,   feature_col, BATCH_SIZE, shuffle=False, num_classes=num_classes)
        test_gen  = SingleFeatureDataGenerator(test_df,  feature_col, BATCH_SIZE, shuffle=False, num_classes=num_classes)

        # Model
        model = create_simple_model(input_shape, num_classes, model_name=feature_type)
        model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

        history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=[early_stopping])
        history_list.append(history.history)

        # Evaluation
        loss, acc = model.evaluate(test_gen)
        loss_list.append(loss)
        accuracy_list.append(acc)
        print(f"{feature_type} - Fold {fold+1} Test accuracy: {acc:.4f}")

        # Predictions & Reports
        y_pred = model.predict(test_gen)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = []
        for _, labels in test_gen:
            y_true.extend(np.argmax(labels, axis=1))
        y_true = np.array(y_true)

        report = classification_report(y_true, y_pred_classes, output_dict=True)
        classification_reports.append(report)
        conf_matrix = confusion_matrix(y_true, y_pred_classes).tolist()
        confusion_matrices.append(conf_matrix)

        # Save model
        os.makedirs(f"models/{feature_type}", exist_ok=True)
        model.save(f"models/{feature_type}/model_fold{fold+1}.h5")

    # Save results
    results[feature_type] = {
        "accuracy_list": accuracy_list,
        "loss_list": loss_list,
        "histories": history_list,
        "classification_reports": classification_reports,
        "confusion_matrices": confusion_matrices,
    }

    with open(f"models/{feature_type}/results.json", "w") as f:
        json.dump(results[feature_type], f, indent=2)

print("\nAll training complete. Models and results saved in 'models/'")

Training features:   0%|          | 0/10 [00:00<?, ?it/s]


Training model for mel_spectrogram

--- Fold 1/5 ---
Epoch 1/200
18/18 [==============================] - 0s 13ms/step - loss: 24.2725 - accuracy: 0.4531 - val_loss: 8.8255 - val_accuracy: 0.5833
Epoch 2/200
18/18 [==============================] - 0s 9ms/step - loss: 5.5070 - accuracy: 0.8125 - val_loss: 4.1611 - val_accuracy: 0.5625
Epoch 3/200
18/18 [==============================] - 0s 9ms/step - loss: 2.6623 - accuracy: 0.8767 - val_loss: 1.9545 - val_accuracy: 0.7986
Epoch 4/200
18/18 [==============================] - 0s 9ms/step - loss: 0.8001 - accuracy: 0.9392 - val_loss: 2.0552 - val_accuracy: 0.8125
Epoch 5/200
18/18 [==============================] - 0s 9ms/step - loss: 1.0501 - accuracy: 0.9566 - val_loss: 2.4383 - val_accuracy: 0.8264
Epoch 6/200
18/18 [==============================] - 0s 9ms/step - loss: 0.4653 - accuracy: 0.9722 - val_loss: 2.3321 - val_accuracy: 0.8472
Epoch 7/200
18/18 [==============================] - 0s 9ms/step - loss: 0.4563 - accuracy: 0.9653

/opt/conda/envs/myenv/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


18/18 [==============================] - 0s 12ms/step - loss: 25.0009 - accuracy: 0.4688 - val_loss: 10.0489 - val_accuracy: 0.4375
Epoch 2/200
18/18 [==============================] - 0s 9ms/step - loss: 5.3601 - accuracy: 0.8090 - val_loss: 8.8720 - val_accuracy: 0.5000
Epoch 3/200
18/18 [==============================] - 0s 9ms/step - loss: 3.6684 - accuracy: 0.8872 - val_loss: 3.0500 - val_accuracy: 0.7708
Epoch 4/200
18/18 [==============================] - 0s 9ms/step - loss: 0.9208 - accuracy: 0.9549 - val_loss: 4.0983 - val_accuracy: 0.7500
Epoch 5/200
18/18 [==============================] - 0s 9ms/step - loss: 0.7747 - accuracy: 0.9583 - val_loss: 3.7998 - val_accuracy: 0.8194
Epoch 6/200
18/18 [==============================] - 0s 9ms/step - loss: 0.3047 - accuracy: 0.9792 - val_loss: 6.6249 - val_accuracy: 0.7222
Epoch 7/200
18/18 [==============================] - 0s 9ms/step - loss: 0.2280 - accuracy: 0.9861 - val_loss: 3.7514 - val_accuracy: 0.7847
Epoch 8/200
18/18 [===

/opt/conda/envs/myenv/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

18/18 [==============================] - 0s 13ms/step - loss: 24.4031 - accuracy: 0.5660 - val_loss: 6.8925 - val_accuracy: 0.5903
Epoch 2/200
18/18 [==============================] - 0s 10ms/step - loss: 4.2467 - accuracy: 0.8681 - val_loss: 3.6317 - val_accuracy: 0.7917
Epoch 3/200
18/18 [==============================] - 0s 10ms/step - loss: 1.1867 - accuracy: 0.9444 - val_loss: 1.4868 - val_accuracy: 0.8889
Epoch 4/200
18/18 [==============================] - 0s 10ms/step - loss: 0.6486 - accuracy: 0.9635 - val_loss: 1.7285 - val_accuracy: 0.9167
Epoch 5/200
18/18 [==============================] - 0s 10ms/step - loss: 0.1896 - accuracy: 0.9896 - val_loss: 1.3162 - val_accuracy: 0.9375
Epoch 6/200
18/18 [==============================] - 0s 9ms/step - loss: 0.0965 - accuracy: 0.9948 - val_loss: 1.5846 - val_accuracy: 0.9167
Epoch 7/200
18/18 [==============================] - 0s 10ms/step - loss: 0.0829 - accuracy: 0.9931 - val_loss: 1.6048 - val_accuracy: 0.9306
Epoch 8/200
18/18 

/opt/conda/envs/myenv/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
Training features:  70%|███████   | 7/10 [02:59<01:18, 26.04s/it]


Training model for stft

--- Fold 1/5 ---
Epoch 1/200


Training features:  70%|███████   | 7/10 [03:00<01:17, 25.75s/it]


InvalidArgumentError: Graph execution error:

Detected at node stft/stft_output/BiasAdd defined at (most recent call last):
  File "/opt/conda/envs/myenv/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/opt/conda/envs/myenv/lib/python3.9/runpy.py", line 87, in _run_code

  File "/opt/conda/envs/myenv/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/opt/conda/envs/myenv/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/conda/envs/myenv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/opt/conda/envs/myenv/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/opt/conda/envs/myenv/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/opt/conda/envs/myenv/lib/python3.9/asyncio/base_events.py", line 1905, in _run_once

  File "/opt/conda/envs/myenv/lib/python3.9/asyncio/events.py", line 80, in _run



In [ ]:
# Create ensemble predictions
print("\n" + "="*50)
print("Creating Ensemble Predictions")
print("="*50)

# Use the last fold of each feature type for ensemble evaluation
ensemble_results = {
    'accuracy_list': [],
    'loss_list': [],
    'classification_reports': [],
    'confusion_matrices': []
}

# For simplicity, we'll use the last fold of each feature type
for fold in tqdm(range(KFOLD_SPLITS), desc = "Ensemble Folds", Leave = True):
    print(f"\n--- Ensemble Fold {fold + 1}/{KFOLD_SPLITS} ---")
    
    # Get predictions from all models for this fold
    all_predictions = {}
    
    for feature_type in available_feature_types:
        if feature_type in all_results:
            # Get the model from this fold
            model = all_results[feature_type]['models'][fold]
            
            # Get test data for this fold (we need to recreate it)
            df = df_dict[feature_type]
            kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42)
            train_idx, test_idx = list(kf.split(df))[fold]
            test_df = df.iloc[test_idx].reset_index(drop=True)
            
            test_generator = SingleFeatureDataGenerator(test_df, feature_type, batch_size=BATCH_SIZE, shuffle=False)
            
            # Get predictions
            pred = model.predict(test_generator, verbose=0)
            all_predictions[feature_type] = pred
            
            # Store true labels (should be the same for all feature types)
            if 'y_true' not in locals():
                y_true = test_generator.get_labels()
    
    # Simple averaging ensemble
    if all_predictions:
        ensemble_pred = np.mean(list(all_predictions.values()), axis=0)
        ensemble_pred_classes = np.argmax(ensemble_pred, axis=1)
        
        # Calculate ensemble accuracy
        ensemble_accuracy = accuracy_score(y_true, ensemble_pred_classes)
        ensemble_results['accuracy_list'].append(ensemble_accuracy)
        
        print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")
        
        # Classification report
        report = classification_report(y_true, ensemble_pred_classes, output_dict=True)
        ensemble_results['classification_reports'].append(report)
        
        # Confusion matrix
        conf_matrix = confusion_matrix(y_true, ensemble_pred_classes).tolist()
        ensemble_results['confusion_matrices'].append(conf_matrix)

# Store ensemble results
all_results['ensemble'] = ensemble_results

In [ ]:
# Save results and models
try:
    os.mkdir("ensemble_models")
except FileExistsError:
    print("Folder already exists")
except Exception:
    print("Unknown error")

# Create version folder
date_part = datetime.now().date().__str__().replace('-', '_')
last_version = os.listdir(path="ensemble_models") if os.path.exists("ensemble_models") else []
last_version = [name.rpartition("_v")[-1] for name in last_version if date_part in name]
if len(last_version):
    last_version = int(sorted(last_version)[-1])
else:
    last_version = 0
folder_name = f"{date_part}_v{last_version+1}"

os.makedirs(os.path.join("ensemble_models", folder_name), exist_ok=True)

# Save individual models
for feature_type, model in all_models.items():
    model_path = os.path.join("ensemble_models", folder_name, f"{feature_type}_model.h5")
    model.save(model_path)
    print(f"Saved {feature_type} model to {model_path}")

# Save results
results_data = {
    'individual_results': {ft: {k: v for k, v in res.items() if k != 'models'} 
                          for ft, res in all_results.items() if ft != 'ensemble'},
    'ensemble_results': all_results['ensemble'],
    'feature_types': available_feature_types,
    'num_classes': num_classes,
    'feature_shapes': FEATURE_SHAPES,
    'training_config': {
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'kfold_splits': KFOLD_SPLITS,
        'fixed_length': FIXED_LENGTH
    },
    'instrument_mappings': instruments_mappings.to_dict()
}

results_path = os.path.join("ensemble_models", folder_name, "results.json")
with open(results_path, 'w') as f:
    json.dump(results_data, f, indent=2, default=str)

print(f"\nResults saved to: {results_path}")
print(f"Models saved to: ensemble_models/{folder_name}/")

In [ ]:
# Print summary of results
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)

print("\nIndividual Model Performance:")
for feature_type in available_feature_types:
    if feature_type in all_results:
        accuracies = all_results[feature_type]['accuracy_list']
        mean_acc = np.mean(accuracies)
        std_acc = np.std(accuracies)
        print(f"  {feature_type}: {mean_acc:.4f} ± {std_acc:.4f}")

print("\nEnsemble Performance:")
if 'ensemble' in all_results:
    ensemble_accuracies = all_results['ensemble']['accuracy_list']
    ensemble_mean = np.mean(ensemble_accuracies)
    ensemble_std = np.std(ensemble_accuracies)
    print(f"  Ensemble: {ensemble_mean:.4f} ± {ensemble_std:.4f}")

# Find best individual model
best_individual = max(
    [(ft, np.mean(all_results[ft]['accuracy_list'])) 
     for ft in available_feature_types if ft in all_results],
    key=lambda x: x[1]
)

improvement = ensemble_mean - best_individual[1]
print(f"\nBest Individual Model: {best_individual[0]} ({best_individual[1]:.4f})")
print(f"Ensemble Improvement: {improvement:.4f} ({improvement*100:.2f}%)")